# Hello World, "Cytomat Incubator"!

The Cytomat series of incubators is used for storing microplates under
controlled environmental conditions. PyLabRobot implements the
{class}`~pylabrobot.incubators.cytomat.cytomat.CytomatBackend` which
supports several models such as `C6000`, `C6002`, `C2C_50`, `C2C_425`,
`C2C_450_SHAKE` and `C5C`.

In this tutorial we show how to:
- connect to the incubator
- configure racks
- move plates in and out
- monitor temperature and humidity

```{note}
This notebook uses `await` statements which must be run inside an
asynchronous environment such as `asyncio`.
```

In [ ]:
from pylabrobot.incubators import CytomatBackend, CytomatType
from pylabrobot.incubators.cytomat.racks import cytomat_rack_9mm_51
from pylabrobot.incubators.incubator import Incubator
from pylabrobot.resources.corning.plates import Cor_96_wellplate_360ul_Fb
from pylabrobot.resources import Coordinate

# Connect to the incubator via a serial port
backend = CytomatBackend(model=CytomatType.C6000, port="/dev/ttyUSB0")

# Create a rack and assemble an `Incubator` resource
rack = cytomat_rack_9mm_51("rack_A")
incubator = Incubator(
    backend=backend,
    name="cyto",
    size_x=860,
    size_y=550,
    size_z=900,
    racks=[rack],
    loading_tray_location=Coordinate(0, 0, 0),
)

## Setup

Setting up the incubator opens the serial connection and initializes the
device.

In [ ]:
await incubator.setup()

## Storing a plate

To store a plate we first place it on the loading tray and then call
{meth}`~pylabrobot.incubators.incubator.Incubator.take_in_plate`.
You can choose a site automatically or specify one explicitly.


In [ ]:
plate = Cor_96_wellplate_360ul_Fb("my_plate")
incubator.loading_tray.assign_child_resource(plate)
await incubator.take_in_plate("smallest")  # choose the smallest free site

# other options:
# await incubator.take_in_plate("random")  # random free site
# await incubator.take_in_plate(rack[3])  # store at rack position 3


## Retrieving a plate

Use {meth}`~pylabrobot.incubators.incubator.Incubator.fetch_plate_to_loading_tray`
to move a plate from storage to the loading tray.

In [ ]:
await incubator.fetch_plate_to_loading_tray("my_plate")
retrieved = incubator.loading_tray.resource

## Monitoring conditions

The Cytomat provides queries for temperature and humidity.

In [ ]:
current_temp = await incubator.get_temperature()
current_humidity = await incubator.backend.get_humidity()
print(current_temp, current_humidity)

## Shutdown

Always close the connection when finished.

In [ ]:
await incubator.stop()